In [1]:
import ROOT
from ROOT import TMVA, TCut, TFile
from math import floor

Welcome to JupyROOT 6.22/03


In [2]:
outputFile = TFile( "TMVA.root", 'RECREATE' )
TMVA.Tools.Instance()

factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

In [3]:
factory.SetVerbose()

Ora dichiariamo un DataLoader e aggiungiamo le variabili usando la AddVariable Function

In [4]:
dataloader = TMVA.DataLoader("dataset")

In [5]:
dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "tImpParV0", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "nSigmaTOFpr", 'F' )
dataloader.AddVariable( "nSigmaTOFpi", 'F' )
dataloader.AddVariable( "nSigmaTOFka", 'F' )
dataloader.AddVariable( "nSigmaTPCpr", 'F' )
dataloader.AddVariable( "nSigmaTPCpi", 'F' )
dataloader.AddVariable( "nSigmaTPCka", 'F' )

#spectator variables

dataloader.AddSpectator( "massLc2K0Sp", "mass Lc-->K0S+p", 'F' )
dataloader.AddSpectator( "LcPt",  "Lc Pt", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  "mass Lc -->L(1520)+pi", 'F' )
dataloader.AddSpectator( "massLambda", "mass V0 = Lambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", "mass V0 = LambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", "V0 positive Pt", 'F' )
dataloader.AddSpectator( "V0negativePt", "V0 negative Pt", 'F' )
dataloader.AddSpectator( "dcaV0pos", "dca V0 positive", 'F' )
dataloader.AddSpectator( "dcaV0neg", "dca V0 negative", 'F' )
dataloader.AddSpectator( "v0Pt", "K0S Pt", 'F' )
dataloader.AddSpectator( "dcaV0", "dca V0", 'F' )
dataloader.AddSpectator( "V0positiveEta", "V0pos Eta", 'F' )
dataloader.AddSpectator( "bachelorEta", "eta bachelor", 'F' )

apriamo il dataset contenente signal and background trees

In [6]:
input= TFile.Open( "AnalysisResults.root")

Warning in <TClass::Init>: no dictionary for class AliRDHFCutsLctoV0 is available
Warning in <TClass::Init>: no dictionary for class AliRDHFCuts is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisCuts is available
Warning in <TClass::Init>: no dictionary for class AliVCuts is available
Warning in <TClass::Init>: no dictionary for class AliESDtrackCuts is available
Warning in <TClass::Init>: no dictionary for class AliAODPidHF is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisUtils is available
Warning in <TClass::Init>: no dictionary for class AliEventCuts is available
Warning in <TClass::Init>: no dictionary for class AliNormalizationCounter is available
Warning in <TClass::Init>: no dictionary for class AliCounterCollection is available


In [7]:
# signal and backg trees for training
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights) Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test.

In [8]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [9]:
mycuts = TCut("LcPt<8 && LcPt>6")
mycutb = TCut("LcPt<8 && LcPt>6")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree


In [10]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
print(nTrainingEventsSgn)
print(nTrainingEventsBkg)
print(nTestingEventsSgn)
print(nTestingEventsBkg)

100000
126478
100000
126478


Usiamo la funzione DataLoader.PrepareTrainingAndTestTree per applicare i cut agli eventi di input

In [11]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")


In [12]:
factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDTG","!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2" );

Factory                  : Booking method: BDTG
                         : 
                         : the option NegWeightTreatment=InverseBoostNegWeights does not exist for BoostType=Grad
                         : --> change to new default NegWeightTreatment=Pray
                         : Building event vectors for type 2 Signal
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Sgn
                         : Building event vectors for type 2 Background
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Bkg
DataSetFactory           : [dataset] : Number of events in input trees
                         : Dataset[dataset] :     Signal     requirement: "LcPt<8 && LcPt>6"
                         : Dataset[dataset] :     Signal          -- number of events passed: 218418  / sum of weights: 218418
                         : Dataset[dataset] :     Signal          -- efficiency             : 0.1239

In [13]:
#factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP", "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

In [14]:
# Train MVAs
factory.TrainAllMethods()

Factory                  : Train all methods
Factory                  : [dataset] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'massK0S' <---> Output : variable 'massK0S'
                         : Input : variable 'tImpParBach' <---> Output : variable 'tImpParBach'
                         : Input : variable 'tImpParV0' <---> Output : variable 'tImpParV0'
                         : Input : variable 'CtK0S' <---> Output : variable 'CtK0S'
                         : Input : variable 'cosPAK0S' <---> Output : variable 'cosPAK0S'
                         : Input : variable 'CosThetaStar' <---> Output : variable 'CosThetaStar'
                         : Input : variable 'nSigmaTOFpr' <---> Output : variable 'nSigmaTOFpr'
                         : Input : variable 'nSigmaTOFpi' <---> Output : variable 'nSigmaTOFpi'
                         : 

0%, time left: unknown
6%, time left: 108 sec
12%, time left: 96 sec
18%, time left: 88 sec
25%, time left: 82 sec
31%, time left: 74 sec
37%, time left: 67 sec
43%, time left: 60 sec
50%, time left: 53 sec
56%, time left: 46 sec
62%, time left: 40 sec
68%, time left: 33 sec
75%, time left: 27 sec
81%, time left: 20 sec
87%, time left: 13 sec
93%, time left: 6 sec
0%, time left: unknown
6%, time left: 17 sec
12%, time left: 13 sec
18%, time left: 11 sec
25%, time left: 9 sec
31%, time left: 8 sec
37%, time left: 7 sec
43%, time left: 6 sec
50%, time left: 5 sec
56%, time left: 5 sec
62%, time left: 4 sec
68%, time left: 3 sec
75%, time left: 2 sec
81%, time left: 2 sec
87%, time left: 1 sec
93%, time left: 0 sec


In [15]:
# Test MVAs
factory.TestAllMethods()

Factory                  : Test all methods
Factory                  : Test method: BDTG for Classification performance
                         : 
BDTG                     : [dataset] : Evaluation of BDTG on testing sample (143222 events)
                         : Elapsed time for evaluation of 143222 events: 10.9 sec       


0%, time left: unknown
6%, time left: 6 sec
12%, time left: 5 sec
18%, time left: 5 sec
25%, time left: 5 sec
31%, time left: 5 sec
37%, time left: 4 sec
43%, time left: 4 sec
50%, time left: 3 sec
56%, time left: 4 sec
62%, time left: 4 sec
68%, time left: 3 sec
75%, time left: 2 sec
81%, time left: 1 sec
87%, time left: 1 sec
93%, time left: 0 sec


In [16]:
# Evaluate MVAs
factory.EvaluateAllMethods() 

Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: BDTG
                         : 
BDTG                     : [dataset] : Loop over test events and fill histograms with classifier response...
                         : 
TFHandler_BDTG           :     Variable            Mean            RMS    [        Min            Max ]
                         : -------------------------------------------------------------------------------
                         :      massK0S:       0.49804     0.0038542   [       0.48762       0.50761 ]
                         :  tImpParBach:   -0.00017059      0.034713   [      -0.49536       0.49891 ]
                         :    tImpParV0:    0.00050277      0.097152   [       -1.4894        1.4882 ]
                         :        CtK0S:        2.8075        2.5434   [      0.046159        132.48 ]
                         :     cosPAK0S:       0.99988    0.00061402   [       0.99001        1.0000 ]
        

In [17]:
# Save the output.
outputFile.Close()